In [1]:
import torch
from torch import nn


通常在cpu上给权重模型初始化好，然后移到GPU上计算
输入命令：nvidia-smi

## 计算设备
默认情况下，张量是在内存中创建的，然后使用CPU计算它。

In [5]:
print(torch.device('cpu'))
print(torch.device('cuda'))
print(torch.device('cuda:1'))

cpu
cuda
cuda:1


## 查询GPU数量

In [ ]:
print(torch.cuda.device_count())

In [7]:
def try_gpu(i=0):
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
def try_all_gpus():
    devices = [torch.device(f"cuda:{i}") for i in
               range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

print(try_gpu(),try_gpu(10),try_all_gpus())

cuda:0 cpu [device(type='cuda', index=0)]


## 张量与GPU
我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

In [8]:
x = torch.tensor([1,2,3])
print(x.device)


cpu


## 存储在GPU上
有几种方法可以在GPU上存储张量。 例如，我们可以在创建张量时指定存储设备。接 下来，我们在第一个gpu上创建张量变量X。 在GPU上创建的张量只消耗这个GPU的显存。 我们可以使用nvidia-smi命令查看显存使用情况。 一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [9]:
X = torch.ones(2,3,device=try_gpu())
print(X)



tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


## 旁注
人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。 但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。 这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收）， 然后才能继续进行更多的操作。 这就是为什么拷贝操作要格外小心。 根据经验，多个小操作比一个大操作糟糕得多。 此外，一次执行几个操作比代码中散布的许多单个操作要好得多（除非你确信自己在做什么）。 如果一个设备必须等待另一个设备才能执行其他操作， 那么这样的操作可能会阻塞。 这有点像排队订购咖啡，而不像通过电话预先订购： 当你到店的时候，咖啡已经准备好了。

## 神经网络模型与GPU
神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上。

In [10]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

In [12]:
print(net(X))

tensor([[-0.9078],
        [-0.9078]], device='cuda:0', grad_fn=<AddmmBackward0>)


确认参数在GPU上

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)